In [ ]:
# 10/19/2021
# No need to run first section again. 
# Run import statements then proceed to Preprocess Data section where csv is loaded in.

# BINNING NEEDS WORK IN ORDER FOR MODELS TO RUN

!pip uninstall scikit-learn -y

!pip install -U scikit-learn

# !pip uninstall xgboost -y

# !pip install -U xgboost

In [1]:
# import dependencies
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
# from config import Password, Username, Host,DB
import pandas as pd

# Preprocess Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [138]:
movies_df = pd.read_csv("/content/drive/MyDrive/final project/dummy_data_frame.csv")

In [139]:
movies_df.head()

,movie_id,rating,date,runtime,metascore,imdb_rating,boxoffice,budget,buzz,Adventure,Comedy,Family,Animation,Action,Sci-Fi,Mystery,Thriller,Biography,Drama,Sport,Fantasy,Crime,Romance,Music,Musical,Horror,History,War,Documentary,News,Western,Short,Unnamed: 32
0,1,PG,2006-12-22,108,48,6.4,250863268,110000000,6,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,G,2006-06-09,117,73,7.1,244082982,70000000,4,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,PG-13,2006-05-26,104,58,6.7,234362462,210000000,23,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,PG-13,2006-05-19,149,46,6.6,217536138,125000000,7,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,PG-13,2006-06-28,154,72,6.0,200120000,232000000,15,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [140]:
len(movies_df.loc[movies_df["budget"]<2000000])

129

In [141]:
len(movies_df.loc[(movies_df["budget"]>2000000) & (movies_df["budget"]<10000000)])

277

In [142]:
len(movies_df.loc[(movies_df["budget"]>10000000) & (movies_df["budget"]<20000000)])

251

In [143]:
len(movies_df.loc[(movies_df["budget"]>20000000)])

840

In [144]:
labels = ["20M", "100M", "250M","800M"]
breaks = [0, 20000000, 100000000, 200000000,800000000]
target = pd.cut(movies_df["boxoffice"] , bins=breaks, labels=labels, include_lowest=True)

In [14]:
# separate out target: Box Office
# bin target data
!pip install jenkspy
import jenkspy
labels_15 = ["0-10724389", "10724389-27367660", "27367660-46462469","46462469-67790117","67790117-92186262","92186262-120146040", 
            "120146040-153636354", "153636354-191719337", "191719337-228778661", "228778661-268488329","268488329-312433331", 
            "312433331-368065385", "368065385-459005868", "459005868-533720947", "533720947-760507625"]
labels_3 = ["65M", "202M", "760M"]
labels_4 = ["42M","121M","262M","760M"]
breaks = jenkspy.jenks_breaks(movies_df["boxoffice"], nb_class=3)
target = pd.cut(movies_df["boxoffice"] , bins=breaks, labels=labels_3, include_lowest=True)


     |████████████████████████████████| 55 kB 2.3 MB/s 
  Created wheel for jenkspy: filename=jenkspy-0.2.0-cp37-cp37m-linux_x86_64.whl size=70411 sha256=34b2d1d65a3cead10250a9e7eba2821428ae2e6f6cc24c1a2c8ecb76b7a4584e
  Stored in directory: /root/.cache/pip/wheels/d8/d1/42/9bd05164ef706b84674905892f0542f8db6ac4a652473ca7e0
Successfully built jenkspy


In [116]:
breaks

[0, 20000000, 100000000, 200000000, 800000000]

In [145]:

movies_df.drop(columns=["boxoffice"], inplace=True)

In [146]:
# clean data
# date into month
movies_df["month"] = pd.DatetimeIndex(movies_df['date']).month


In [147]:
# make dummies for rating
dummy_df = pd.get_dummies(movies_df, columns=["rating"])

In [148]:
dummy_df.drop(columns=["movie_id", "date"], inplace=True)

In [149]:
# split data into test train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dummy_df, target, random_state=1)

In [150]:
# scale data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create Models

In [151]:
# random forest
# need to bin data for classifications
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(random_state=1, n_estimators=1000).fit(X_train_scaled, y_train)

In [152]:
print(f'Training Score: {random_forest_model.score(X_train_scaled, y_train)}')
print(f'Testing Score: {random_forest_model.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 0.6258992805755396


In [24]:
# neural network
import tensorflow as tf
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=100, activation="sigmoid", input_dim=40))

nn_model.add(tf.keras.layers.Dense(units=100, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=100, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 24,401
Trainable params: 24,401
Non-trainable params: 0
_________________________________________________________________


In [25]:
# neural net
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# neural net
# Fit the model to the training data
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [ ]:

fit_model = nn_model.fit(X_train, le.transform(y_train), epochs=200)

In [ ]:
import sklearn
sklearn.__version__

'0.22.2.post1'

In [157]:
# xg boost
from xgboost import XGBClassifier

# fit model no training data
xg_model = XGBClassifier()
xg_model.fit(X_train_scaled, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [158]:
# xg boost
# make predictions for test data
y_pred = xg_model.predict(X_test_scaled)
predictions = [value for value in y_pred]

from sklearn.metrics import accuracy_score
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 63.07%


# Output Models

In [162]:
import pickle
# pickle.dump(xg_model, open("/content/drive/MyDrive/final project/new_xg_model.pkl", 'wb'))
# pickle.dump(random_forest_model, open("/content/drive/MyDrive/final project/new_rf_model.pkl", 'wb'))

# from keras.models import load_model

# nn_model.save('/content/drive/MyDrive/final project/nn_model.h5')

In [ ]:
from keras.models import load_model
nn_model = load_model('/content/drive/MyDrive/final project/nn_model.h5')

In [31]:
import numpy as np


In [153]:
testing_data = [[155,75.0,8.4,165000000,9,1,0,0.0,0,1,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,0,1,0,0.0,0.0,0.0,0.0],
[106,68.0,6.3,21000000,4,1,1,0.0,1,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,1,0,0,0.0,0.0,0.0,0.0],
[106,98.0,8.0,21000000,4,0,0,0.0,0,0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0],
[111,85.0,6.4,21000000,4,0,0,0.0,0,0,0.0,0.0,0.0,1,1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0],
[110,69.0,6.2,21000000,4,0,0,0.0,0,0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0],
[108,78.0,7.3,21000000,4,0,0,0.0,0,0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0],
[138,78.0,7.0,21000000,4,0,1,0.0,0,0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0],
[105,66.0,7.0,21000000,4,0,0,0.0,0,0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0]]
# testing_data = np.asarray(testing_data).astype('float32')
scaler = StandardScaler().fit(testing_data)
testing_data_scaled = scaler.transform(testing_data)

In [38]:
nn_model.predict(testing_data_scaled)

array([[0.8544535 ],
       [0.85540247],
       [0.85525024],
       [0.85570097],
       [0.8552783 ],
       [0.8552118 ],
       [0.8552035 ],
       [0.8552247 ]], dtype=float32)

In [159]:
xg_model.predict(testing_data_scaled)

array(['800M', '100M', '20M', '20M', '100M', '100M', '100M', '100M'],
      dtype=object)

In [160]:
xg_model.feature_importances_

array([0.02770951, 0.0322915 , 0.02753296, 0.15736097, 0.02829025,
       0.03403816, 0.03223614, 0.0226501 , 0.04746706, 0.02572445,
       0.02857441, 0.02514708, 0.02550575, 0.04485691, 0.02777937,
       0.03154815, 0.        , 0.03965484, 0.02758227, 0.02120727,
       0.01564781, 0.04135492, 0.02366539, 0.01815636, 0.01343245,
       0.        , 0.        , 0.        , 0.        , 0.03246538,
       0.03145334, 0.        , 0.03020088, 0.00526138, 0.03371483,
       0.02703991, 0.        , 0.        , 0.        , 0.02045021],
      dtype=float32)

In [154]:
random_forest_model.predict(testing_data_scaled)

array(['100M', '100M', '20M', '20M', '20M', '20M', '20M', '100M'],
      dtype=object)

In [155]:
random_forest_model.feature_importances_

array([9.86207331e-02, 1.00056359e-01, 9.39728302e-02, 2.91917611e-01,
       8.15941787e-02, 1.30634289e-02, 1.73698154e-02, 7.11580106e-03,
       6.84783447e-03, 1.31908515e-02, 7.13440374e-03, 9.08598409e-03,
       1.19021560e-02, 1.11474375e-02, 1.98006169e-02, 5.51425245e-03,
       8.38852027e-03, 1.29996861e-02, 1.37968978e-02, 6.63767346e-03,
       2.31009438e-03, 7.98353852e-03, 5.57426521e-03, 3.41147682e-03,
       7.10265732e-03, 6.87652408e-04, 1.02514978e-03, 3.97703594e-04,
       1.95620190e-03, 7.76287915e-02, 3.14855768e-03, 8.70402122e-05,
       1.06685084e-02, 8.82935929e-03, 1.59134810e-02, 1.65763290e-02,
       7.32654694e-04, 7.79448601e-04, 3.42411910e-04, 4.68760657e-03])

In [163]:

pickle.dump(random_forest_model, open("/content/drive/MyDrive/final project/best_rf_model.pkl", 'wb'))
pickle.dump(xg_model, open("/content/drive/MyDrive/final project/best_xg_model.pkl", 'wb'))